# Механизми на внимание и трансформери

Един от основните недостатъци на рекурентните мрежи е, че всички думи в последователността имат еднакво влияние върху резултата. Това води до подоптимална производителност при стандартните LSTM модели за енкодер-декодер за задачи от тип последователност към последователност, като разпознаване на именувани обекти и машинен превод. В действителност, определени думи в входната последователност често имат по-голямо влияние върху изходните последователности от други.

Да разгледаме модел от тип последователност към последователност, като машинен превод. Той се реализира чрез две рекурентни мрежи, където едната мрежа (**енкодер**) компресира входната последователност в скрито състояние, а другата, **декодер**, разгъва това скрито състояние в преведен резултат. Проблемът с този подход е, че крайното състояние на мрежата трудно запомня началото на изречението, което води до ниско качество на модела при дълги изречения.

**Механизмите на внимание** предоставят начин за претегляне на контекстуалното влияние на всеки входен вектор върху всяка изходна прогноза на RNN. Това се реализира чрез създаване на преки връзки между междинните състояния на входната RNN и изходната RNN. По този начин, при генериране на изходен символ $y_t$, ще вземем предвид всички входни скрити състояния $h_i$, с различни теглови коефициенти $\alpha_{t,i}$.

![Изображение, показващо модел енкодер/декодер с добавен слой за внимание](../../../../../translated_images/bg/encoder-decoder-attention.7a726296894fb567.webp)
*Моделът енкодер-декодер с механизъм за добавено внимание в [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), цитиран от [този блог пост](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Матрицата на внимание $\{\alpha_{i,j}\}$ представлява степента, до която определени входни думи играят роля в генерирането на дадена дума в изходната последователност. По-долу е даден пример за такава матрица:

![Изображение, показващо пример за подравняване, намерено от RNNsearch-50, взето от Bahdanau - arviz.org](../../../../../translated_images/bg/bahdanau-fig3.09ba2d37f202a6af.webp)

*Фигура, взета от [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Фиг.3)*

Механизмите на внимание са отговорни за голяма част от настоящите или почти настоящите върхови постижения в обработката на естествен език. Добавянето на внимание обаче значително увеличава броя на параметрите на модела, което води до проблеми със скалирането при RNN. Основно ограничение при скалирането на RNN е, че рекурентната природа на моделите затруднява групирането и паралелизирането на обучението. В RNN всеки елемент от последователността трябва да бъде обработен в последователен ред, което означава, че не може лесно да бъде паралелизиран.

Приемането на механизми на внимание, комбинирано с това ограничение, доведе до създаването на съвременните трансформер модели, които познаваме и използваме днес, от BERT до OpenGPT3.

## Трансформер модели

Вместо да предават контекста на всяка предишна прогноза в следващата стъпка на оценка, **трансформер моделите** използват **позиционни кодировки** и **внимание**, за да уловят контекста на даден вход в предоставен прозорец от текст. Изображението по-долу показва как позиционните кодировки с внимание могат да уловят контекста в даден прозорец.

![Анимиран GIF, показващ как се извършват оценките в трансформер моделите.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Тъй като всяка входна позиция се съпоставя независимо с всяка изходна позиция, трансформерите могат да се паралелизират по-добре от RNN, което позволява създаването на много по-големи и по-изразителни езикови модели. Всяка глава на внимание може да се използва за изучаване на различни взаимоотношения между думите, което подобрява задачите за обработка на естествен език.

## Създаване на прост трансформер модел

Keras не съдържа вградени слоеве за трансформери, но можем да създадем свои собствени. Както и преди, ще се фокусираме върху текстова класификация на набора от данни AG News, но си струва да се отбележи, че трансформер моделите показват най-добри резултати при по-сложни задачи за обработка на естествен език.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Новите слоеве в Keras трябва да наследяват класа `Layer` и да имплементират метода `call`. Нека започнем със слоя **Positional Embedding**. Ще използваме [някакъв код от официалната документация на Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Ще приемем, че запълваме всички входни последователности до дължина `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Този слой се състои от два слоя `Embedding`: за вграждане на токени (по начина, който обсъдихме по-рано) и за вграждане на позициите на токените. Позициите на токените се създават като последователност от естествени числа от 0 до `maxlen`, използвайки `tf.range`, и след това се подават през слоя за вграждане. Двата получени векторa за вграждане се събират, като се получава позиционно вградено представяне на входа с форма `maxlen`$\times$`embed_dim`.

Сега, нека имплементираме блока на трансформъра. Той ще използва изхода от предварително дефинирания слой за вграждане:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Сега сме готови да дефинираме пълния модел на трансформър:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer Models

**BERT** (Bidirectional Encoder Representations from Transformers) е много голяма многослойна трансформаторна мрежа с 12 слоя за *BERT-base* и 24 за *BERT-large*. Моделът първо се предварително обучава върху голям корпус от текстови данни (WikiPedia + книги) чрез обучение без надзор (предсказване на маскирани думи в изречение). По време на предварителното обучение моделът усвоява значително ниво на езиково разбиране, което след това може да бъде използвано с други набори от данни чрез фина настройка. Този процес се нарича **трансферно обучение**.

![картина от http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/bg/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Съществуват много вариации на трансформаторни архитектури, включително BERT, DistilBERT, BigBird, OpenGPT3 и други, които могат да бъдат фино настроени.

Нека видим как можем да използваме предварително обучен модел BERT за решаване на нашия традиционен проблем с класификация на последователности. Ще заимстваме идеята и част от кода от [официалната документация](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

За да заредим предварително обучени модели, ще използваме **Tensorflow hub**. Първо, нека заредим специфичния за BERT векторизатор:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Важно е да използвате същия векторизатор, който е бил използван за обучение на оригиналната мрежа. Освен това, BERT векторизаторът връща три компонента:
* `input_word_ids`, което е последователност от номера на токени за входното изречение
* `input_mask`, показваща коя част от последователността съдържа действителния вход и коя е запълване. Това е подобно на маската, произведена от слоя `Masking`
* `input_type_ids` се използва за задачи по езиково моделиране и позволява да се специфицират две входни изречения в една последователност.

След това можем да създадем екстрактор на характеристики BERT:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Така че, BERT слой връща няколко полезни резултата:
* `pooled_output` е резултат от осредняване на всички токени в последователността. Можете да го разглеждате като интелигентно семантично вграждане на цялата мрежа. Той е еквивалентен на изхода на слоя `GlobalAveragePooling1D` в предишния ни модел.
* `sequence_output` е изходът на последния трансформър слой (съответства на изхода на `TransformerBlock` в модела ни по-горе).
* `encoder_outputs` са изходите на всички трансформър слоеве. Тъй като сме заредили BERT модел с 4 слоя (както вероятно можете да предположите от името, което съдържа `4_H`), той има 4 тензора. Последният е същият като `sequence_output`.

Сега ще дефинираме крайния модел за класификация. Ще използваме *функционално дефиниране на модела*, при което дефинираме входа на модела и след това предоставяме серия от изрази за изчисляване на неговия изход. Също така ще направим теглата на BERT модела не-трениращи и ще тренираме само крайния класификатор:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Въпреки че има малко параметри за обучение, процесът е доста бавен, защото екстракторът на характеристики на BERT е изчислително тежък. Изглежда, че не успяхме да постигнем разумна точност, или поради липса на обучение, или поради недостатъчни параметри на модела.

Нека опитаме да отключим теглата на BERT и да го обучим също. Това изисква много малка скорост на обучение, както и по-внимателна стратегия за обучение с **затопляне**, използвайки оптимизатора **AdamW**. Ще използваме пакета `tf-models-official`, за да създадем оптимизатора:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Както виждате, обучението върви доста бавно - но може да искате да експериментирате и да обучите модела за няколко епохи (5-10), за да видите дали можете да постигнете по-добър резултат в сравнение с подходите, които сме използвали преди.

## Библиотека Huggingface Transformers

Друг много разпространен (и малко по-опростен) начин за използване на Transformer модели е [пакетът HuggingFace](https://github.com/huggingface/), който предоставя лесни за използване модули за различни NLP задачи. Той е достъпен както за Tensorflow, така и за PyTorch, друга много популярна рамка за невронни мрежи.

> **Note**: Ако не се интересувате как работи библиотеката Transformers - можете да пропуснете до края на този ноутбук, защото няма да видите нещо съществено различно от това, което направихме по-горе. Ще повтаряме същите стъпки за обучение на BERT модел, използвайки различна библиотека и значително по-голям модел. Следователно, процесът включва доста дълго обучение, така че може просто да искате да прегледате кода.

Нека видим как нашият проблем може да бъде решен с помощта на [Huggingface Transformers](http://huggingface.co).


Първото нещо, което трябва да направим, е да изберем модела, който ще използваме. Освен някои вградени модели, Huggingface разполага с [онлайн хранилище за модели](https://huggingface.co/models), където можете да намерите още много предварително обучени модели, създадени от общността. Всички тези модели могат да бъдат заредени и използвани само чрез предоставяне на името на модела. Всички необходими двоични файлове за модела ще бъдат автоматично изтеглени.

В определени случаи може да се наложи да заредите свои собствени модели. В този случай можете да посочите директорията, която съдържа всички свързани файлове, включително параметри за токенизатора, файла `config.json` с параметрите на модела, двоичните тегла и т.н.

От името на модела можем да създадем както модела, така и токенизатора. Нека започнем с токенизатора:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Обектът `tokenizer` съдържа функцията `encode`, която може да се използва директно за кодиране на текст:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Можем също да използваме токенизатор, за да кодираме последователност по начин, подходящ за подаване към модела, т.е. включително полетата `token_ids`, `input_mask` и др. Можем също да уточним, че искаме тензори на Tensorflow, като предоставим аргумента `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

В нашия случай ще използваме предварително обучен BERT модел, наречен `bert-base-uncased`. *Uncased* означава, че моделът е нечувствителен към главни и малки букви.

Когато обучаваме модела, трябва да предоставим токенизирана последователност като вход, и затова ще проектираме процес за обработка на данни. Тъй като `tokenizer.encode` е Python функция, ще използваме същия подход, както в последния модул, като я извикаме с помощта на `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Сега можем да заредим действителния модел, използвайки пакета `BertForSequenceClassification`. Това гарантира, че нашият модел вече има необходимата архитектура за класификация, включително крайния класификатор. Ще видите предупредително съобщение, което посочва, че теглата на крайния класификатор не са инициализирани и моделът ще изисква предварително обучение - това е напълно нормално, защото точно това ще направим!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Както можете да видите от `summary()`, моделът съдържа почти 110 милиона параметри! Предполага се, че ако искаме проста задача за класификация върху сравнително малък набор от данни, не искаме да обучаваме основния слой на BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Сега сме готови да започнем обучението!

> **Забележка**: Обучението на пълномащабен BERT модел може да отнеме много време! Затова ще го обучим само за първите 32 партиди. Това е само за да покажем как се настройва обучението на модела. Ако искате да опитате пълномащабно обучение - просто премахнете параметрите `steps_per_epoch` и `validation_steps` и се подгответе да чакате!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Ако увеличите броя на итерациите, изчакате достатъчно дълго и тренирате за няколко епохи, можете да очаквате, че класификацията с BERT ще ни даде най-добрата точност! Това е така, защото BERT вече разбира доста добре структурата на езика и ние трябва само да донастроим крайния класификатор. Въпреки това, тъй като BERT е голям модел, целият процес на обучение отнема много време и изисква сериозна изчислителна мощност! (GPU, и за предпочитане повече от един).

> **Note:** В нашия пример използваме един от най-малките предварително обучени модели на BERT. Съществуват по-големи модели, които вероятно ще дадат по-добри резултати.


## Основни моменти

В този модул разгледахме най-новите архитектури на модели, базирани на **transformers**. Приложихме ги за нашата задача за класификация на текст, но по същия начин моделите BERT могат да се използват за извличане на обекти, отговаряне на въпроси и други задачи в областта на обработката на естествен език (NLP).

Моделите на трансформъри представляват най-съвременното постижение в NLP и в повечето случаи трябва да бъдат първото решение, с което започвате да експериментирате при внедряване на персонализирани NLP решения. Въпреки това, разбирането на основните принципи на рекурентните невронни мрежи, обсъдени в този модул, е изключително важно, ако искате да изграждате усъвършенствани невронни модели.



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
